## Graph Attention Network (GAT)


### 1.1 Importing Dependencies

We import the necessary libraries and functions, ensuring that all required modules and helper functions are properly integrated.

In [1]:
import networkx as nx
from torch_geometric.utils import from_networkx
import os
import sys

# gat → models → src
src_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if src_path not in sys.path:
    sys.path.append(src_path)


import import_ipynb 
from utils.wrapper.transform_networkx_into_pyg import transform_networkx_into_pyg
from utils.add_dummy_node_features import add_dummy_node_features


### 1.2 Loading and Preparing Graph Data from GraphML Files

This code snippet loads a series of bicycle traffic network graphs stored in GraphML format and prepares them for training with PyTorch Geometric (PyG). The objective is to convert each monthly graph into a format compatible with Graph Neural Networks (GNNs), ensuring that edge features are retained.

Each NetworkX graph is converted into a PyG `Data` object using a custom helper function `networkx_to_pyg`. This function ensures that essential edge attributes such as:

- `tracks` (the number of bicycles traveling from the starting to the ending point),
- `month` and `year`,
- `speed_rel` (relative speed),

are preserved during the conversion process.

PyG expects data in a specific structure, particularly when edge attributes are used in models like GATv2.

`data_list` contains multiple `torch_geometric.data.Data` objects, each representing a graph.

**NOTE:** So far I only did for data from 2023. I will iterate through all available data, when the implementation is finished.


In [2]:

# Initialize an empty list to store the PyTorch Geometric Data objects
data_list = []

# Iterate over the 12 graph files (from 0 to 11)
for i in range(12):  # 0 to 11
    # Build the path to the graph file
    path = f"../../../graphs/2023/bike_network_2023_{i}.graphml"
    
    # Read the graph from the GraphML file
    G_nx = nx.read_graphml(path)
    
    # Ensure the graph is loaded as a directed graph (DiGraph)
    G_nx = nx.DiGraph(G_nx)
    
    # Use the custom function to convert the NetworkX graph to a PyTorch Geometric Data object
    # The edge attributes (such as 'tracks', 'month', 'year', 'speed_rel') will be preserved
    data = transform_networkx_into_pyg(G_nx)
    
    # Append the Data object to the list
    data_list.append(data)

# Check the result - number of graphs (Data objects) loaded
print(f"Number of graphs: {len(data_list)}")


Number of graphs: 12


### 1.3 Adding Dummy Node Features to the Graphs

In this section of the code, we add **dummy node features** to our graphs. This process ensures that each node in our graphs has a **feature dimension**, even if no node features were originally present. This is an important step in preparing the data for use in Graph Neural Networks (GNNs).

**NOTE:** At a later stage, once we have implemented feature engineering, we will replace the dummy features with the engineered ones.


In [3]:
data_list = add_dummy_node_features(data_list, feature_dim=1, value=1.0)


### 1.4 Train-Validation Split

For predicting edge attributes (e.g., `tracks`), an 80/20 train/validation split was applied to the **existing edges within each graph**.

In our application, the **nodes represent physically existing bike stations**, which typically do not change or only change very infrequently. The aim of the analysis is to model the **connections between stations**, i.e., to understand and predict how many bicycles move along certain routes (in other words: edges with weights).

A **node-level split** (i.e., an 80/20 split of the nodes themselves) would mean that some stations would be completely unseen during training. This would not be meaningful because:

- The **stations themselves are not the prediction target**;
- It is the **relationships or transitions between the stations (edges)** that should be modeled;
- In deployment, **all stations are known** (they are physically installed in the system);

**NOTE:** GCN and GCN-GRU could also use these functions. So we would move them to the folder utils


In [4]:
from torch_geometric.transforms import RandomLinkSplit

# Define the 80/20 train/val split
transform = RandomLinkSplit(
    num_val=0.2,  # 20% for validation
    num_test=0.0,  # no test set (test set will be 2024 data)
    is_undirected=False,  # Set to False if your graphs are directed
    split_labels=False,
)

# Apply the transform to each graph in your list
train_val_data_list = [transform(data) for data in data_list]

# Now you get a list of (train_data, val_data) tuples
for i, (train_data, val_data, _) in enumerate(train_val_data_list):
    print(f"Graph {i}:")
    print(f"  Train edges: {train_data.edge_index.size(1)}")
    print(f"  Val edges:   {val_data.edge_label_index.size(1)}")



Graph 0:
  Train edges: 16671
  Val edges:   8334
Graph 1:
  Train edges: 17980
  Val edges:   8988
Graph 2:
  Train edges: 18066
  Val edges:   9032
Graph 3:
  Train edges: 22253
  Val edges:   11126
Graph 4:
  Train edges: 26821
  Val edges:   13410
Graph 5:
  Train edges: 38612
  Val edges:   19304
Graph 6:
  Train edges: 31458
  Val edges:   15728
Graph 7:
  Train edges: 30864
  Val edges:   15432
Graph 8:
  Train edges: 30796
  Val edges:   15396
Graph 9:
  Train edges: 24759
  Val edges:   12378
Graph 10:
  Train edges: 19477
  Val edges:   9738
Graph 11:
  Train edges: 14930
  Val edges:   7464


## 2. Implementing a GAT Model

### 2.1 From GAT to GATv2

We initially started by implementing a vanilla GAT model—an advanced type of Graph Neural Network (GNN) that leverages **attention mechanisms**. However, we soon realized that this model is not capable of learning from **edge attributes**, which is essential for our task. This limitation became especially critical because our original dataset does not contain any **node attributes** at all.

This led us to adopt the **GATv2** model, which is specifically designed to aggregate node features while also considering **edge attributes** during message passing. It is more suitable for our purposes.

The GATv2 model expects the following **inputs**:
- `x`: Node features → `data.x`
- `edge_index`: Edge list → `data.edge_index`
- `edge_attr`: Edge attributes → `data.edge_attr`

These inputs are automatically passed from the `Data` object when calling the model.

**Output:**  
The model returns **node representations (embeddings)**—a tensor with one row per node and one column per output feature.



In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv


class GATv2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, edge_dim, heads=1):
        super(GATv2, self).__init__()

        # First GATv2 layer, with edge attributes
        self.gat1 = GATv2Conv(in_channels, hidden_channels, heads=heads, edge_dim=edge_dim)

        # Second GATv2 layer, output dimension = out_channels
        self.gat2 = GATv2Conv(hidden_channels * heads, out_channels, heads=1, edge_dim=edge_dim)

    def forward(self, x, edge_index, edge_attr):
        # Apply first GATv2 layer with edge attributes
        x = self.gat1(x, edge_index, edge_attr)
        x = F.elu(x)

        # Apply second GATv2 layer
        x = self.gat2(x, edge_index, edge_attr)
        return x


### 2.2 Advancing to an Encoder-Decoder Architecture

However, the original GATv2 model is primarily designed to learn **node embeddings**. These are useful for tasks such as node classification but are **not directly applicable to predicting edge attributes** like our target edge weight `tracks`.

Since our objective is to **predict edge values**, using a node-only model is insufficient. To address this, we extend the GATv2 architecture by incorporating a **decoder module** that transforms node embeddings into edge-level predictions.

Our final model follows a typical **encoder-decoder architecture**:

- **Encoder:**  
  We use the GATv2 model as the encoder to compute **informative node embeddings** based on the graph structure, edge attributes, and (if available) node features.

- **Decoder:**  
  As the decoder, we use a **small multilayer perceptron (MLP)** that takes as input the **concatenated embeddings** of each edge's source and target nodes.  
  This MLP outputs a **single scalar value per edge**, which serves as the prediction for the edge attribute `tracks`.


In [6]:
import torch.nn as nn

class GATv2EdgePredictor(nn.Module):
    def __init__(self, 
                 in_channels, 
                 hidden_channels, 
                 out_channels, 
                 edge_dim, 
                 heads=1):
        super(GATv2EdgePredictor, self).__init__()

        # 1. GATv2 model for computing node embeddings
        self.gnn = GATv2(in_channels, hidden_channels, out_channels, edge_dim, heads)

        # 2. Edge MLP to predict edge attributes (e.g., "tracks")
        self.edge_mlp = nn.Sequential(
            nn.Linear(out_channels * 2, out_channels),
            nn.ReLU(),
            nn.Linear(out_channels, 1)  # Output: a single scalar per edge
        )

    def forward(self, data):
        """
        Args:
            data: PyTorch Geometric Data object with attributes:
                  - x: node features
                  - edge_index: edge connectivity (COO format)
                  - edge_attr: edge attributes

        Returns:
            pred: Tensor of shape [num_edges, 1] with predicted edge weights (e.g., "tracks")
        """
        # Compute node embeddings using the GATv2 model
        x = self.gnn(data.x, data.edge_index, data.edge_attr)  # [num_nodes, out_channels]

        # Construct edge representations by concatenating source and target node embeddings
        row, col = data.edge_index  # source & target node indices for each edge
        edge_inputs = torch.cat([x[row], x[col]], dim=1)  # [num_edges, out_channels * 2]

        # Predict edge weights
        pred = self.edge_mlp(edge_inputs)  # [num_edges, 1]
        return pred


## 3. Hyperparamter Optimization 


### 3.1 Setting Random Seeds and Creating Directory for Models and Configurations

To ensure the reproducibility of the experiments, random seeds are set for both PyTorch and Python (`torch.manual_seed(42)` and `random.seed(42)`). This guarantees that the results remain consistent across different runs of the code. Additionally, a directory named `hpo_models` is created where models and configuration files will be saved.

In [7]:
import os
import random
import json

# Set random seeds for reproducibility
torch.manual_seed(42)
random.seed(42)

# Create directory for saving models and configs
os.makedirs("hpo_models", exist_ok=True)

### 3.2 Defining the Hyperparameter Search Space and Training Settings

The hyperparameter search space is defined as a set of possible values for several key hyperparameters, including:
- the learning rate (`lr`),
- the number of hidden units in intermediate layers (`hidden_channels`),
- the size of the output features (`out_channels`),
- and the number of attention heads (`heads`) used in the GATv2 model.

This search space can be expanded or modified depending on the model's requirements. For example:
- **Optimizer**: We currently use Adam, but alternatives like SGD or AdamW could also be tested.
- **Loss function**: MSELoss is used here, but other functions such as L1Loss could be considered.
- **Dropout**: This could be applied within GATv2 or in the edge-level MLP, although it is not used in the current implementation.
- **Batch size**: Currently, training is performed on the entire graph (no mini-batching). However, support for mini-batching could be implemented.

In addition to defining the search space, the training and validation settings are specified as follows:
- The number of hyperparameter optimization (HPO) trials (`num_trials`) is set to 20.
- Each model is trained for a maximum of 1000 epochs (`num_epochs`).
- Early stopping is applied with a patience of 5 epochs (`early_stopping_patience`), meaning training halts if the validation loss does not improve over five consecutive epochs.

The best-performing configuration and the lowest validation loss encountered across all trials are saved for later use. Lastly, the index of the target edge attribute to be predicted (e.g., `"tracks"`) is explicitly set.


In [8]:
# Define hyperparameter search space (can be extended or modified)
search_space = {
    "lr": [1e-1, 5e-2, 1e-2, 1e-3, 5e-4, 1e-4],  # Learning rates
    "hidden_channels": [8, 16, 32, 64],          # Hidden layer sizes
    "out_channels": [8, 16, 32],                 # Output feature sizes from GNN
    "heads": [1, 2, 4],                          # Number of attention heads in GATv2
    # "dropout": [0.0, 0.1, 0.3, 0.5],           # Dropout
}

# Define number of HPO trials and training settings
num_trials = 20
num_epochs = 1000
early_stopping_patience = 5  # Number of epochs to wait for improvement
best_overall_val_loss = float("inf")
best_config = None

# Index of the target edge attribute to predict (e.g., "tracks" at column 4)
target_idx = 4



### 3.3 Hyperparameter Optimization with Random Search

This section performs a randomized hyperparameter search across a predefined space. For each trial, a set of hyperparameters is sampled and used to instantiate a new `GATv2EdgePredictor` model. The model is trained using the Adam optimizer and Mean Squared Error (MSE) loss for a maximum of `num_epochs` iterations. At the end of all trials, the best overall model and its corresponding configuration are saved for future use.

In [9]:
# Begin hyperparameter search loop
for trial in range(num_trials):
    # Randomly sample a configuration from the search space
    config = {
        "lr": random.choice(search_space["lr"]),
        "hidden_channels": random.choice(search_space["hidden_channels"]),
        "out_channels": random.choice(search_space["out_channels"]),
        "heads": random.choice(search_space["heads"]),
        # "dropout": random.choice(search_space["dropout"]),
    }

    print(f"\n Trial {trial+1}/{num_trials} | Config: {config}")

    # Instantiate model with current configuration
    model = GATv2EdgePredictor(
        in_channels=train_data.num_node_features,
        hidden_channels=config["hidden_channels"],
        out_channels=config["out_channels"],
        edge_dim=train_data.edge_attr.shape[1],
        heads=config["heads"]
    )

    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
    criterion = torch.nn.MSELoss()

    best_val_loss = float("inf")
    epochs_without_improvement = 0

    # Training loop
    for epoch in range(1, num_epochs + 1):
        model.train()
        optimizer.zero_grad()

        pred = model(train_data)
        target = train_data.edge_attr[:, target_idx].unsqueeze(1)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()

        # Print training loss periodically
        if epoch % 10 == 0 or epoch == 1:
            print(f"Epoch {epoch:03d} | Training Loss: {loss.item():.4f}")

        # Evaluate on validation set every 50 epochs
        if epoch % 50 == 0:
            model.eval()
            with torch.no_grad():
                val_pred = model(val_data)
                val_target = val_data.edge_attr[:, target_idx].unsqueeze(1)
                val_loss = criterion(val_pred, val_target)
            print(f"Epoch {epoch:03d} | Validation Loss: {val_loss.item():.4f}")

            # Save the model if it improves the best validation loss
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                epochs_without_improvement = 0
            else:
                epochs_without_improvement += 1

            # Apply early stopping if no improvement
            if epochs_without_improvement >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch}")
                break

    print(f"Trial {trial+1} complete | Best Validation Loss: {best_val_loss:.4f}")

    # Update global best model and config if current trial is better
    if best_val_loss < best_overall_val_loss:
        best_overall_val_loss = best_val_loss
        best_config = {
            **config,               # original hyperparameters
            "trial": trial + 1,     # current trial number (1-based)
            "val_loss": best_val_loss.item()  # best validation loss
        }
        torch.save(model.state_dict(), "hpo_models/best_model_overall.pth")
        with open("hpo_models/best_config_overall.json", "w") as f:
            json.dump(best_config, f, indent=2)  # nicely formatted for readability


print("\nRandom search completed.")
print(f"Best configuration: {best_config}")
print(f"Best validation loss: {best_overall_val_loss:.4f}")



 Trial 1/20 | Config: {'lr': 0.0001, 'hidden_channels': 8, 'out_channels': 8, 'heads': 4}
Epoch 001 | Training Loss: 308.1389
Epoch 010 | Training Loss: 307.6332
Epoch 020 | Training Loss: 307.0688
Epoch 030 | Training Loss: 306.5000
Epoch 040 | Training Loss: 305.9235
Epoch 050 | Training Loss: 305.3930
Epoch 050 | Validation Loss: 305.3380
Epoch 060 | Training Loss: 304.8385
Epoch 070 | Training Loss: 304.2743
Epoch 080 | Training Loss: 303.6881
Epoch 090 | Training Loss: 303.0814
Epoch 100 | Training Loss: 302.4483
Epoch 100 | Validation Loss: 302.3837
Epoch 110 | Training Loss: 301.7864
Epoch 120 | Training Loss: 301.0919
Epoch 130 | Training Loss: 300.3627
Epoch 140 | Training Loss: 299.5991
Epoch 150 | Training Loss: 298.7982
Epoch 150 | Validation Loss: 298.7162
Epoch 160 | Training Loss: 297.9598
Epoch 170 | Training Loss: 297.1698
Epoch 180 | Training Loss: 296.3419
Epoch 190 | Training Loss: 295.4757


KeyboardInterrupt: 